In [1]:

import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Masking
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import torch


In [26]:
df=pd.read_csv('./data/data_6.csv')

In [27]:
def transform_data(sel,names):
  
  datamain=sel.query("SYMBOL=='%s'"%(names[0]))
  datamain=datamain[["DATE","PRICE"]]
  datamain.drop_duplicates(subset=["DATE"], keep='first', inplace=True)
  datamain=datamain.sort_values(by="DATE")
  s=names[0]+"_price"
  s2=names[0]+"_size"
  # datamain["DATE"]=pd.to_datetime(datamain["DATE"])
  datamain=datamain.rename(columns={"PRICE":s})
  # datamain[s] = pd.to_numeric(datamain["PRICE"])
  # datamain[s2] = pd.to_numeric(datamain["SIZE"])


  for name in names:
    if name==names[0]:
      continue
    data=sel.query("SYMBOL=='%s'"%name)

    data=data[["DATE","PRICE"]]
    data.drop_duplicates(subset=["DATE"], keep='first', inplace=True)
    
    # data["PRICE"]=pd.to_datetime(data["PRICE"])
    # data["Close_"] = pd.to_numeric(data["Close_"])
 
    data=data.rename(columns={"PRICE": "%s_price"%name})


    datamain=pd.merge(datamain,data,on="DATE", how="outer")
  return datamain

In [31]:
names=list(set(df["SYMBOL"]))
df=transform_data(df,names)

In [4]:
def mape(A,F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
   
    if maskf_sub[i]!=0:
      sum+=abs(A[i] - F[i]) / abs(A[i])
      length+=1
  if length>0:
    return 100/length*sum
  
  return 0
def smape(A, F,maskf_sub):
  sum=0
  length=0
  for i in range(len(A)):
    if maskf_sub[i]!=0:
      sum+=2 * np.abs(F[i] - A[i]) / (np.abs(A[i]) + np.abs(F[i]))
      length+=1
  if length>0:

    return 100/length * sum
  return 0


def get_mask(data):
    """
    data should in the form of pd.df
    gen a tenor with 0 and 1 to represent missing data
    """
    mask = ~data.isna().values
 
    mask_tensor = torch.tensor(mask, dtype=torch.float32)
    
    mask_tensor= mask_tensor.transpose(0,1)
    return mask_tensor


def normalize(data):
    first_list=[]
    for i in data.columns:
       
        first=data[i].iloc[0]
        p=1
        
        while np.isnan(first):
           first=data[i].iloc[p]

           p+=1
        
        data[i]=(data[i])/first
        first_list.append(first)   
    return data,first_list

In [13]:

def lstm_run(df,name,ep,j,num_fea):
    """df is the data in pd.DataFrame
    name is the list of columns name to be predicted
    ep is the epoch to train
    j is the prediction length"""    
    
    X = df[name][:-1]
    Y = df[name][1:]
  

    X_norm,X_div =normalize(X) 
    Y_norm,Y_div =normalize(Y)
    Y_test_mask=get_mask(Y_norm)[:,-j:]
    X_norm=X_norm.fillna(0)
    Y_norm=Y_norm.fillna(0)
    X_norm=X_norm.values
    Y_norm=Y_norm.values

    X_train, Y_train = X_norm[:-j], Y_norm[:-j]
    X_test, Y_test = X_norm[-j:], Y_norm[-j:]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    model = Sequential()
    model.add(Masking(mask_value=0,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(X_train.shape[2]))  
    model.compile(loss='mae', optimizer='adam')
    model.fit(X_train, Y_train, epochs=ep, batch_size=32, validation_data=(X_test, Y_test), verbose=2, shuffle=False)

    test_predict = model.predict(X_test)
    smpl=[]
    mpl=[]
    pred=[]
    test=[]
    pred=test_predict
    test=Y_test

    Y_test=Y_test[-j:].transpose(1,0)
    test_predict=test_predict[-j:].transpose(1,0)
    print(Y_test.shape,test_predict.shape)
    print(Y_div[0])


    for i in range(num_fea):

        mp=mape(Y_test[i]*Y_div[i],test_predict[i]*Y_div[i],Y_test_mask[i])
        smp=smape(Y_test[i]*Y_div[i],test_predict[i]*Y_div[i],Y_test_mask[i])
        pred[:,i]=pred[:,i]*Y_div[i]
        test[:,i]=test[:,i]*Y_div[i]
        smpl.append(smp)
        mpl.append(mp)
    return pred,test,mpl,smpl

In [36]:
ep=10
j=1
name=df.columns[1:]
num_fea=3
pred,test,mpl,smpl=lstm_run(df,name,ep,j,num_fea)


Epoch 1/10
8/8 - 10s - loss: 0.7348 - val_loss: 0.5937 - 10s/epoch - 1s/step
Epoch 2/10
8/8 - 0s - loss: 0.6813 - val_loss: 0.5447 - 77ms/epoch - 10ms/step
Epoch 3/10
8/8 - 0s - loss: 0.6273 - val_loss: 0.4937 - 80ms/epoch - 10ms/step
Epoch 4/10
8/8 - 0s - loss: 0.5805 - val_loss: 0.4536 - 93ms/epoch - 12ms/step
Epoch 5/10
8/8 - 0s - loss: 0.5367 - val_loss: 0.4178 - 165ms/epoch - 21ms/step
Epoch 6/10
8/8 - 0s - loss: 0.4906 - val_loss: 0.3757 - 187ms/epoch - 23ms/step
Epoch 7/10
8/8 - 0s - loss: 0.4409 - val_loss: 0.3289 - 169ms/epoch - 21ms/step
Epoch 8/10
8/8 - 0s - loss: 0.3863 - val_loss: 0.2771 - 180ms/epoch - 22ms/step
Epoch 9/10
8/8 - 0s - loss: 0.3259 - val_loss: 0.2212 - 155ms/epoch - 19ms/step
Epoch 10/10
8/8 - 0s - loss: 0.2774 - val_loss: 0.2085 - 131ms/epoch - 16ms/step
1/1 [==============================] - 3s 3s/step
(3, 1) (3, 1)
37.16


In [37]:
result= pd.DataFrame({'lstm_mape':mpl,'lstm_smape':smpl})
result.to_csv('D:/timeseries/result/temp_2.csv')
res2= pd.DataFrame({'name':name,'day1/1_pred':pred[0], 'day1/1_actu':test[0]
                       })
res2.to_csv('D:/timeseries/result/temp.csv')

In [ ]:
result= pd.DataFrame({'lstm_mape':mpl,'lstm_smape':smpl})
result.to_csv('D:/timeseries/result/temp_2.csv')
res2= pd.DataFrame({'name':name,'day1/7_pred':pred[0], 'day1/7_actu':test[0],'day2/7_pred':pred[1], 
                    'day2/7_actu':test[1],'day3/7_pred':pred[2], 'day3/7_actu':test[2],'day4/7_pred':pred[3], 
                    'day4/7_actu':test[3],'day5/7_pred':pred[4], 'day5/7_actu':test[4],'day6/7_pred':pred[5],
                      'day6/7_actu':test[5],'day7/7_pred':pred[6], 'day7/7_actu':test[6]
                       })
res2.to_csv('D:/timeseries/result/temp.csv')